<a href="https://colab.research.google.com/github/OE-LUCIFER/youtube-video/blob/main/Finetune/Finetune_LLM_for_prompt_gen_for_txt_to_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[torch]
!pip install datasets
!pip install accelerate -U
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from datasets import load_dataset

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("mrm8488/bloom-560m-finetuned-sd-prompts")

# Set the padding token to the EOS token
tokenizer.pad_token = tokenizer.eos_token

# Load dataset from Hugging Face
# dataset = load_dataset('UnfilteredAI/Prompts-NSFW')['train']
dataset = load_dataset('Gustavosta/Stable-Diffusion-Prompts')['train']
# Tokenize the dataset
def tokenize_function(examples):
    if "prompt" in examples:
        text_to_tokenize = examples["prompt"]
    elif "Prompt" in examples:
        text_to_tokenize = examples["Prompt"]
    else:
        raise KeyError("Missing 'prompt' or 'Prompt' key in examples")
    return tokenizer(text_to_tokenize, padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Set the format of the dataset
tokenized_dataset.set_format(type='torch', columns=['input_ids'])

# Initialize the model
model = AutoModelForCausalLM.from_pretrained("mrm8488/bloom-560m-finetuned-sd-prompts")

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=1, # Adjust this value based on your needs
    per_device_train_batch_size=1,
    save_steps=100,
    save_total_limit=1,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    ),
    train_dataset=tokenized_dataset,
)

# Start training
trainer.train()


In [2]:
model.save_pretrained("promptgen-nsfw")

In [3]:
tokenizer.save_pretrained("promptgen-nsfw")

('promptgen-nsfw/tokenizer_config.json',
 'promptgen-nsfw/special_tokens_map.json',
 'promptgen-nsfw/tokenizer.json')

In [4]:
model.push_to_hub("UnfilteredAI/promptgen-nsfw")
tokenizer.push_to_hub("UnfilteredAI/promptgen-nsfw")

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/UnfilteredAI/promptgen-nsfw/commit/50f89ca7171bf62abea67f40146d921505442b33', commit_message='Upload tokenizer', commit_description='', oid='50f89ca7171bf62abea67f40146d921505442b33', pr_url=None, pr_revision=None, pr_num=None)

```<s>Prompt: pikachu dinning in the eiffel tower```